# Process DICOM from XNAT to Linked Dicom TTL

### import necessary components

In [1]:
#ORAW modifications and integrations with XNAT by Leonard Wee during Oct -> Dec 2021

from __future__ import print_function

from time import process_time
import os
import subprocess
import glob
import shutil
import pandas as pd
from datetime import date, datetime
from numpy.random import choice

import xnat #needed for reading and writing to/from XNAT
from DicomDatabase import DicomDatabase # dicom indexer by Johan van Soest

import LinkedDicom

### define needed operations

In [2]:
def upload_file(session, project, subject, experiment, assessment, resource, convfile):
    xnat_project = session.projects[project]
    xnat_subject = session.classes.SubjectData(parent=xnat_project, label=subject)
    xnat_experiment = session.classes.CtSessionData(parent=xnat_subject, label=experiment)
    xnat_resource = session.classes.ResourceCatalog(parent=xnat_experiment, label=resource)
    #xnat_assessment = session.classes.QcAssessmentData(parent=xnat_experiment, label=assessment) #not used
    #xnat_resource = session.classes.ResourceCatalog(parent=xnat_assessment, label=resource) #resource under experiment instead
    xnat_resource.upload(convfile, os.path.basename(convfile)) # upload
#    for file_ in data:
#        resource.upload(file_, os.path.basename(file_))
#    pass


def cleanup_temp_folder(dirt):
    if os.path.exists(dirt):
        shutil.rmtree(dirt)
        os.makedirs(dirt)
    if not os.path.exists(dirt):
        os.makedirs(dirt)

### MANDATORY : configure user-dependent settings

In [3]:
#------------------------- USER SETTINGS -------------------------------

# set up XNAT login credentials here
# .......................
#xnatUrl = 'https://xnat.bmia.nl'
#xnatUser = 'leonardwee'
xnatUrl = 'http://xnat-nginx:80'  #change me! e.g. 'http://localhost:8081/'
xnatUser = 'admin'  #change me!
xnatPass = 'admin'  #change me!
xnatProject = 'argos-testbed'  #change me

## NB : This is REQUIRED to initiate a Linked Dicom SCP to send the output triples to the graph database
#subprocess.call(['ldcm-scp', '-s', 'http://rdf-store:7200/repositories/userRepo/statements', '104']) #change third argument to your own graph database API

### Optional : retrieve list of patients and dicom experiments from XNAT project

In [5]:
with xnat.connect(xnatUrl, user=xnatUser, password=xnatPass) as session:
    myProject= session.projects[xnatProject]
    mySubjectsList = myProject.subjects.values()
    for s in mySubjectsList:
        mySubjectID = s.label
        mySubject = myProject.subjects[mySubjectID]
        myExperimentsList = mySubject.experiments.values()
        for e in myExperimentsList:
            myExperimentID = e.label
            myExperiment = mySubject.experiments[myExperimentID]
            print(xnatProject + "\t" + mySubjectID + "\t" + myExperimentID)

[WARNING] Detected a redirect from http://xnat-nginx:80 to http://xnat-nginx/, using http://xnat-nginx/ from now on


argos-testbed	LUNG1-001	LUNG1-001
argos-testbed	LUNG1-002	LUNG1-002
argos-testbed	LUNG1-003	LUNG1-003


### main section

In [6]:
XNATdownload = "./XnatDownload"
RFpath = "./RFStore/RDF_output"

currentResourceLabel = datetime.now().strftime("%Y%m%d_%H%M")

start_time = process_time()

cleanup_temp_folder(XNATdownload) #clears folder if exists, create new if not exist

# -----------------------------------------------------------
with xnat.connect(xnatUrl, user=xnatUser, password=xnatPass) as session:
    myProject= session.projects[xnatProject]
    mySubjectsList = myProject.subjects.values()
    for s in mySubjectsList:
        mySubjectID = s.label
        mySubject = myProject.subjects[mySubjectID]
        myExperimentsList = mySubject.experiments.values()
        for e in myExperimentsList:
            myExperimentID = e.label
            myExperiment = mySubject.experiments[myExperimentID]
            myExperiment.download_dir(XNATdownload)
            print("XNAT object downloaded, subject %s" % mySubjectID)
            ###
            print("Processing %s in Linked Dicom please WAIT ...." % mySubjectID)
            ### -------------- this is for printing out an instance of TTLs to a file; comment out if not required
            subprocess.call(['ldcm-parse','./XNATDownload']) #this directs the contents of this folder to LinkedDicom service
            destination = os.path.join(RFpath,mySubjectID + "_ldcm.ttl")
            try:
                shutil.move('./XNATDownload/linkeddicom.ttl',destination) 
            except:
                print("Error - shutil was unable to move the linkeddicom.ttl output file from Linked Dicom")
            ###
            ### -------------- this is for routing the output TTLs into a graph database; comment out if not required
            subprocess.call(['storescu', '+r', '+sd', '+sp', '*.dcm', 'localhost', '104', './XNATDownload'])
            ###
            print("OK, commencing with next subject in XNAT ....")
##### ----------------------------------------------------------------------------

stop_time = process_time()
print("DONE with processing subjects in XNAT!")
print("Total time elapsed : %s" % (stop_time-start_time))


[WARNING] Detected a redirect from http://xnat-nginx:80 to http://xnat-nginx/, using http://xnat-nginx/ from now on
 68.7 MiB |                                                #      |   2.6 MiB/s


XNAT object downloaded, subject LUNG1-001
Processing LUNG1-001 in Linked Dicom please WAIT ....
Start processing folder ./XNATDownload. Depending on the folder size this might take a while.
Stored results in ./XNATDownload/linkeddicom.ttl
OK, commencing with next subject in XNAT ....


 57.4 MiB |                                                      #|  39.1 KiB/s


XNAT object downloaded, subject LUNG1-002
Processing LUNG1-002 in Linked Dicom please WAIT ....
Start processing folder ./XNATDownload. Depending on the folder size this might take a while.
Stored results in ./XNATDownload/linkeddicom.ttl
OK, commencing with next subject in XNAT ....


 55.2 MiB |               #                                       |   1.5 MiB/s


XNAT object downloaded, subject LUNG1-003
Processing LUNG1-003 in Linked Dicom please WAIT ....
Start processing folder ./XNATDownload. Depending on the folder size this might take a while.
Stored results in ./XNATDownload/linkeddicom.ttl
OK, commencing with next subject in XNAT ....
DONE with processing subjects in XNAT!
Total time elapsed : 108.57964350000003
